<a href="https://colab.research.google.com/github/MYVirajani/Sentiment-Analysis/blob/main/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

#Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 71 bytes


In [ ]:
!kaggle datasets download -d crowdflower/twitter-airline-sentiment

Dataset URL: https://www.kaggle.com/datasets/crowdflower/twitter-airline-sentiment
License(s): CC-BY-NC-SA-4.0
 78% 2.00M/2.55M [00:01<00:00, 2.17MB/s]
100% 2.55M/2.55M [00:01<00:00, 2.30MB/s]


In [ ]:
from zipfile import ZipFile
file_name = "/content/twitter-airline-sentiment.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [ ]:
import pandas as pd

df = pd.read_csv('Tweets.csv')

In [ ]:
df.shape

(14640, 15)

In [ ]:
df = df[["airline_sentiment", "text"]]

In [ ]:
df.head(5)

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


In [ ]:
df.shape

(14640, 2)

In [ ]:
df.groupby('airline_sentiment').describe()

text         \
                  count unique   
airline_sentiment                
negative           9178   9087   
neutral            3099   3067   
positive           2363   2298   

                                                                           
                                                                 top freq  
airline_sentiment                                                          
negative           @AmericanAir that's 16+ extra hours of travel ...    2  
neutral                                           @SouthwestAir sent    5  
positive                                            @JetBlue thanks!    5

In [ ]:
df = df.drop_duplicates(keep='first')

In [ ]:
df.groupby('airline_sentiment').describe()

text         \
                  count unique   
airline_sentiment                
negative           9087   9087   
neutral            3067   3067   
positive           2298   2298   

                                                                           
                                                                 top freq  
airline_sentiment                                                          
negative           @VirginAmerica it's really aggressive to blast...    1  
neutral                          @VirginAmerica What @dhepburn said.    1  
positive           @VirginAmerica plus you've added commercials t...    1

# Preprocessing text

In [ ]:
import nltk
import string
import re

from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')

from nltk.corpus import stopwords

nltk.download('punkt_tab')

ps = PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:

def clean_text(text):

  text = text.lower()

  text = re.sub(r'http.?://[^\s]+[\s]?', '', text)

  text = nltk.word_tokenize(text)

  y = []

  for i in text:

    if i not in stopwords.words('english'):

              y.append(i)

  text = y[:]

  y.clear()

  for i in text:

          y.append(ps.stem(i))

  return" ".join(y)


In [ ]:
df["text_cleaned"]= df["text"].apply(clean_text)

In [ ]:
df.sample(5)

,airline_sentiment,text,text_cleaned
11435,negative,"@USAirways booked an award ticket recently, no...","@ usairway book award ticket recent , ticket l..."
9395,positive,@USAirways made it! Just! Huge relief - thanks...,@ usairway made ! ! huge relief - thank help !
308,positive,@VirginAmerica happy to spend the day together...,@ virginamerica happi spend day togeth . let 's !
10525,negative,@USAirways What's the seat assignment policy? ...,@ usairway 's seat assign polici ? let us choo...
2790,negative,@United super lame that you charge $8 for tv o...,@ unit super lame charg $ 8 tv intern flight u...


# **Feature Extraction**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tf_vec = TfidfVectorizer(max_features=3000) # vocabulary size is 3000, vector size also same
X =tf_vec.fit_transform(df['text_cleaned']).toarray()

In [ ]:
X.shape

(14452, 3000)

In [ ]:
Y= df['airline_sentiment'].values

# **Train Model**

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state = 2)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train,y_train)

MultinomialNB()

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.7388446904185403


In [ ]:
from sklearn.ensemble import RandomForestClassifier


model_02 = RandomForestClassifier()

model_02.fit(X_train,y_train)

y_pred = model_02.predict(X_test)

print(accuracy_score(y_test,y_pred))

0.7699757869249395


In [ ]:
df.head(10)

,airline_sentiment,text,text_cleaned
0,neutral,@VirginAmerica What @dhepburn said.,@ virginamerica @ dhepburn said .
1,positive,@VirginAmerica plus you've added commercials t...,@ virginamerica plu 've ad commerci experi ......
2,neutral,@VirginAmerica I didn't today... Must mean I n...,@ virginamerica n't today ... must mean need t...
3,negative,@VirginAmerica it's really aggressive to blast...,@ virginamerica 's realli aggress blast obnoxi...
4,negative,@VirginAmerica and it's a really big bad thing...,@ virginamerica 's realli big bad thing
5,negative,@VirginAmerica seriously would pay $30 a fligh...,@ virginamerica serious would pay $ 30 flight ...
6,positive,"@VirginAmerica yes, nearly every time I fly VX...","@ virginamerica ye , nearli everi time fli vx ..."
7,neutral,@VirginAmerica Really missed a prime opportuni...,@ virginamerica realli miss prime opportun men...
8,positive,"@virginamerica Well, I didn't…but NOW I DO! :-D","@ virginamerica well , didn't…but ! : -d"
9,positive,"@VirginAmerica it was amazing, and arrived an ...","@ virginamerica amaz , arriv hour earli . 're ..."
